In [1]:
%matplotlib inline

In [1]:
from io import open
import unicodedata
import string
import re
import random

import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
'''!wget https://download.pytorch.org/tutorial/data.zip
!unzip data.zip'''

In [ ]:
'''!tail data/eng-fra.txt'''

In [2]:
SOS_token = 0
EOS_token = 1


class Lang:
    def __init__(self, name):
        self.name = name
        self.word2index = {}
        self.word2count = {}
        self.index2word = {0: "SOS", 1: "EOS"}
        self.n_words = 2  # Count SOS and EOS

    def addSentence(self, sentence):
        for word in sentence.split(' '):
            self.addWord(word)

    def addWord(self, word):
        if word not in self.word2index:
            self.word2index[word] = self.n_words
            self.word2count[word] = 1
            self.index2word[self.n_words] = word
            self.n_words += 1
        else:
            self.word2count[word] += 1

In [3]:
# Turn a Unicode string to plain ASCII, thanks to
# http://stackoverflow.com/a/518232/2809427
def unicodeToAscii(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn'
    )

# Lowercase, trim, and remove non-letter characters


def normalizeString(s):
    s = unicodeToAscii(s.lower().strip())
    s = re.sub(r"([.!?])", r" \1", s)
    s = re.sub(r"[^-a-zA-Z.!?а-яёйЁА-Я]+", r" ", s)
    return s

In [13]:
def readLangs(lang1, lang2, reverse=False):
    print("Reading lines...")

    # Read the file and split into lines
    lines = open('data/%s-%s.txt' % (lang1, lang2), encoding='utf-8').\
        read().strip().split('\n')

    # Split every line into pairs and normalize
    pairs = [[normalizeString(s) for s in l.split('\t')][:2] for l in lines if l !='']

    # Reverse pairs, make Lang instances
    if reverse:
        pairs = [list(reversed(p)) for p in pairs]
        input_lang = Lang(lang2)
        output_lang = Lang(lang1)
    else:
        input_lang = Lang(lang1)
        output_lang = Lang(lang2)

    return input_lang, output_lang, pairs

In [14]:
_,_,p=readLangs('eng', 'rus')
(p[300])

Reading lines...


['go back .', 'иди обратно .']

In [21]:
len(p[444][0])

9

In [19]:
MAX_LENGTH = 10

eng_prefixes = (
    "i am ", "i m ",
    "he is", "he s ",
    "she is", "she s",
    "you are", "you re ",
    "we are", "we re ",
    "they are", "they re "
)


def filterPair(p):
    if len(p)==2:
        return len(p[0].split(' ')) < MAX_LENGTH and \
            len(p[1].split(' ')) < MAX_LENGTH #and p[0].startswith(eng_prefixes)
    else:
        return False
        


def filterPairs(pairs):
    return [pair for pair in pairs  if filterPair(pair)]

In [20]:
def prepareData(lang1, lang2, reverse=False):
    input_lang, output_lang, pairs = readLangs(lang1, lang2, reverse)
    print("Read %s sentence pairs" % len(pairs))
    pairs = filterPairs(pairs)
    print("Trimmed to %s sentence pairs" % len(pairs))
    print("Counting words...")
    for pair in pairs:
        if len(pair)==2:
            input_lang.addSentence(pair[0])
            output_lang.addSentence(pair[1])
    print("Counted words:")
    print(input_lang.name, input_lang.n_words)
    print(output_lang.name, output_lang.n_words)
    return input_lang, output_lang, pairs


input_lang, output_lang, pairs = prepareData('eng', 'rus', True)
print(input_lang, '\n',output_lang, '\n',random.choice(pairs))#



Reading lines...
Read 479223 sentence pairs
Trimmed to 395812 sentence pairs
Counting words...
Counted words:
rus 50735
eng 15710
 ['выбери ракетку которая больше всего тебе нравится .', 'choose your favorite racket .']


In [22]:
for i in range(5):
    print(random.choice(pairs), '\n')

['как мне проити в полицию ?', 'how can i get to the police station ?'] 

['представление почти закончилось .', 'the show s almost over .'] 

['я иногда об этом думаю .', 'i think about it sometimes .'] 

['я бы хотела чтобы вы вернулись в команду .', 'i d like you back on the team .'] 

['у тома раньше был черныи кот .', 'tom used to have a black car .'] 



In [23]:
'''emb = nn.Embedding(11, 1)
# a batch of 2 samples of 4 indices each
input = torch.LongTensor([0,1,2,3,4,5,6,7,8,9,10])
emb(input)'''

'emb = nn.Embedding(11, 1)\n# a batch of 2 samples of 4 indices each\ninput = torch.LongTensor([0,1,2,3,4,5,6,7,8,9,10])\nemb(input)'

The Encoder
-----------





In [24]:
class EncoderRNN(nn.Module):
    def __init__(self, input_size, hidden_size):
        #инициализация при создании объекта класса input_size==n_words, hidden_size=256(размер выходного вектора энкодера)
        super(EncoderRNN, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(input_size, hidden_size)
        self.gru = nn.LSTM(hidden_size, hidden_size,num_layers=2)

    def forward(self, input, hidden):
        #получили одно слово(тензор(-1,1)) из фразы в энкодер ==input_tensor[ei] и hidden==encoder_hidden = encoder.initHidden()
        embedded = self.embedding(input).view(1, 1, -1)
        output = embedded
        output, hidden = self.gru(output, hidden)
        return output, hidden

    def initHidden(self):
        return torch.zeros(2, 1, self.hidden_size, device=device),torch.zeros(2, 1, self.hidden_size, device=device)

The Decoder
-----------




In [25]:
class DecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size):
        super(DecoderRNN, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(output_size, hidden_size)
        self.gru = nn.LSTM(hidden_size, hidden_size,num_layers=2)
        self.out = nn.Linear(hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, input, hidden):
        output = self.embedding(input).view(1, 1, -1)
        output = F.relu(output)
        output, hidden = self.gru(output, hidden)
        output = self.softmax(self.out(output[0]))
        return output, hidden

    def initHidden(self):
        return torch.zeros(2, 1, self.hidden_size, device=device),torch.zeros(2, 1, self.hidden_size, device=device)

In [26]:
def indexesFromSentence(lang, sentence):
    return [lang.word2index[word] for word in sentence.split(' ')]


def tensorFromSentence(lang, sentence):
    indexes = indexesFromSentence(lang, sentence)
    indexes.append(EOS_token)
    return torch.tensor(indexes, dtype=torch.long, device=device).view(-1, 1)


def tensorsFromPair(pair):
    input_tensor = tensorFromSentence(input_lang, pair[0])
    #из фразы получаем тензор размерности [-1,1], где вместо -1 будет количество индексов слов из словаря
    target_tensor = tensorFromSentence(output_lang, pair[1])
    return (input_tensor, target_tensor)

In [105]:
'''encoder_output, encoder_hidden = encoder1(torch.LongTensor([638]), torch.zeros(1, 1, 5, device=device))

encoder_output.shape, encoder_output.squeeze()# или encoder_output[0,0]'''

'encoder_output, encoder_hidden = encoder1(torch.LongTensor([638]), torch.zeros(1, 1, 5, device=device))\n\nencoder_output.shape, encoder_output.squeeze()# или encoder_output[0,0]'

In [27]:
teacher_forcing_ratio = 0.5

#input_tensor и target_tensor тензоры из индексов слов во входной и выходной фразах size==(-1,1) где -1 - индексы слов из словаря
def train(input_tensor, target_tensor, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion, max_length=MAX_LENGTH):
    encoder_hidden = encoder.initHidden()

    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()

    input_length = input_tensor.size(0)  # размер input_tensor и target_tensor имеет право быть разным!
    target_length = target_tensor.size(0)# размер input_tensor и target_tensor имеет право быть разным!

    #encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)

    loss = 0

    for ei in range(input_length):
        #подаём одно слово из фразы(одно чило из тензора) в энкодер ==input_tensor[ei]
        encoder_output, encoder_hidden = encoder(input_tensor[ei], encoder_hidden)
        #получаем выход == output, hidden = self.gru(output, hidden)

        #encoder_outputs[ei] = encoder_output[0, 0] # или encoder_output.squeeze()

    decoder_input = torch.tensor([[SOS_token]], device=device)

    decoder_hidden = encoder_hidden

    use_teacher_forcing = True if random.random() < teacher_forcing_ratio else False

    if use_teacher_forcing:
        # Teacher forcing: Feed the target as the next input
        for di in range(target_length):
            decoder_output, decoder_hidden = decoder(
                decoder_input, decoder_hidden)
            loss += criterion(decoder_output, target_tensor[di])
            decoder_input = target_tensor[di]  # Teacher forcing

    else:
        # Without teacher forcing: use its own predictions as the next input
        for di in range(target_length):
            decoder_output, decoder_hidden = decoder(
                decoder_input, decoder_hidden)
            topv, topi = decoder_output.topk(1)
            decoder_input = topi.squeeze().detach()  # detach from history as input

            loss += criterion(decoder_output, target_tensor[di])
            if decoder_input.item() == EOS_token:
                break

    loss.backward()

    encoder_optimizer.step()
    decoder_optimizer.step()

    return loss.item() / target_length

In [28]:
import time
import math


def asMinutes(s):
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)


def timeSince(since, percent):
    now = time.time()
    s = now - since
    es = s / (percent)
    rs = es - s
    return '%s (- %s)' % (asMinutes(s), asMinutes(rs))

In [108]:
'''tensorsFromPair(random.choice(pairs))[0][3,0]'''

'tensorsFromPair(random.choice(pairs))[0][3,0]'

In [29]:
def trainIters(encoder, decoder, n_iters, print_every=1000, plot_every=100, learning_rate=0.01):
    start = time.time()
    plot_losses = []
    print_loss_total = 0  # Reset every print_every
    plot_loss_total = 0  # Reset every plot_every

    encoder_optimizer = optim.SGD(encoder.parameters(), lr=learning_rate)
    decoder_optimizer = optim.SGD(decoder.parameters(), lr=learning_rate)

    training_pairs = [tensorsFromPair(random.choice(pairs)) for i in range(n_iters)]
    #получили список кортежей вх. и вых. фраз в виде тензоров (-1,1) где -1 - индексы слов из словаря
    criterion = nn.NLLLoss()

    for iter in range(1, n_iters + 1):
        training_pair = training_pairs[iter - 1]
        input_tensor = training_pair[0]#   вх. фраз в виде тензоров (-1,1) где -1 - индексы слов из словаря
        target_tensor = training_pair[1]# вых. фраз в виде тензоров (-1,1) где -1 - индексы слов из словаря

        loss = train(input_tensor, target_tensor, encoder,
                     decoder, encoder_optimizer, decoder_optimizer, criterion)
        print_loss_total += loss
        plot_loss_total += loss

        if iter % print_every == 0:
            print_loss_avg = print_loss_total / print_every
            print_loss_total = 0
            print('%s (%d %d%%) %.4f' % (timeSince(start, iter / n_iters),
                                         iter, iter / n_iters * 100, print_loss_avg))

        if iter % plot_every == 0:
            plot_loss_avg = plot_loss_total / plot_every
            plot_losses.append(plot_loss_avg)
            plot_loss_total = 0

    showPlot(plot_losses)

In [30]:
import matplotlib.pyplot as plt
plt.switch_backend('agg')
import matplotlib.ticker as ticker
import numpy as np


def showPlot(points):
    plt.figure()
    fig, ax = plt.subplots()
    # this locator puts ticks at regular intervals
    loc = ticker.MultipleLocator(base=0.2)
    ax.yaxis.set_major_locator(loc)
    plt.plot(points)

In [31]:
def evaluate(encoder, decoder, sentence, max_length=MAX_LENGTH):
    with torch.no_grad():
        input_tensor = tensorFromSentence(input_lang, sentence)
        input_length = input_tensor.size()[0]
        encoder_hidden = encoder.initHidden()

        encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)

        for ei in range(input_length):
            encoder_output, encoder_hidden = encoder(input_tensor[ei],
                                                     encoder_hidden)
            encoder_outputs[ei] += encoder_output[0, 0]

        decoder_input = torch.tensor([[SOS_token]], device=device)  # SOS

        decoder_hidden = encoder_hidden

        decoded_words = []

        for di in range(max_length):
            decoder_output, decoder_hidden = decoder(
                decoder_input, decoder_hidden)
            topv, topi = decoder_output.data.topk(1)
            if topi.item() == EOS_token:
                decoded_words.append('<EOS>')
                break
            else:
                decoded_words.append(output_lang.index2word[topi.item()])

            decoder_input = topi.squeeze().detach()

        return decoded_words

In [32]:
def evaluateRandomly(encoder, decoder, n=10):
    for i in range(n):
        pair = random.choice(pairs)
        print('>', pair[0])
        print('=', pair[1])
        output_words = evaluate(encoder, decoder, pair[0])
        output_sentence = ' '.join(output_words)
        print('<', output_sentence)
        print('')

In [34]:

hidden_size = 50
encoder1 = EncoderRNN(input_lang.n_words, hidden_size).to(device)
decoder1 = DecoderRNN(hidden_size, output_lang.n_words).to(device)
trainIters(encoder1, decoder1, 200, print_every=10)

0m 2s (- 0m 39s) (10 5%) 9.6650
0m 4s (- 0m 38s) (20 10%) 9.6416
0m 6s (- 0m 36s) (30 15%) 8.9491
0m 8s (- 0m 33s) (40 20%) 7.0665
0m 9s (- 0m 29s) (50 25%) 5.4022
0m 11s (- 0m 27s) (60 30%) 8.6744
0m 13s (- 0m 25s) (70 35%) 6.3202
0m 15s (- 0m 22s) (80 40%) 3.7247
0m 16s (- 0m 20s) (90 45%) 5.3699
0m 18s (- 0m 18s) (100 50%) 5.1709
0m 20s (- 0m 16s) (110 55%) 4.6314
0m 21s (- 0m 14s) (120 60%) 6.0067
0m 23s (- 0m 12s) (130 65%) 6.0711
0m 25s (- 0m 10s) (140 70%) 5.3394
0m 27s (- 0m 9s) (150 75%) 8.2589
0m 28s (- 0m 7s) (160 80%) 5.9067
0m 30s (- 0m 5s) (170 85%) 5.0878
0m 31s (- 0m 3s) (180 90%) 4.2558
0m 33s (- 0m 1s) (190 95%) 4.1737
0m 34s (- 0m 0s) (200 100%) 5.5335


In [35]:
evaluateRandomly(encoder1, decoder1)

> том не учится на своих ошибках .
= tom doesn t learn from his mistakes .
< <EOS>

> я никак не наиду свою сумочку .
= i can t find my purse .
< <EOS>

> я просто хотел чтобы меня оставили в покое .
= i just wanted to be left alone .
< <EOS>

> у меня была та же проблема .
= i ve had the same problem .
< <EOS>

> похоже что-то случилось .
= it seems that something has happened .
< <EOS>

> думаешь том может выиграть ?
= do you think tom can win ?
< <EOS>

> ты когда-нибудь видел как том играет в теннис ?
= have you ever seen tom play tennis ?
< <EOS>

> почему я вам нравлюсь ?
= why do you like me ?
< <EOS>

> я пыталась быть смешнои .
= i was trying to be funny .
< <EOS>

> это мои гражданскии долг .
= it s my civic duty .
< <EOS>

